In [1]:
import votekit.ballot_generator as bg
from votekit.ballot import CumulativeBallot

from votekit.elections import Cumulative
from votekit.pref_profile import PreferenceProfile

# Cumulative Voting

In a cumulative election, voters have $m$ votes to fill $m$ seats. They are allowed to cast these votes on any candidate, including multiple votes for one candidate. Then, the candidates with the $m$ highest vote totals win. We still model this using a preference interval, except now we just sample with replacement (instead of without replacement as in the PL model).

In [7]:
candidates = ["A", "B", "C"]
pref_interval_by_bloc = {"R": {"A": .3, "B":.6, "C":.1}}
bloc_voter_prop = {"R":1}

cumulative = bg.Cumulative(candidates = candidates, pref_interval_by_bloc = pref_interval_by_bloc,
                           bloc_voter_prop=bloc_voter_prop)



In [10]:
cumulative_profile = cumulative.generate_profile(number_of_ballots=100)
cumulative_election = Cumulative(profile = cumulative_profile,
                                 seats = 2)

print(cumulative_profile)

  Ballots  Weight
(B, B, A)      30
(B, B, B)      22
(B, A, A)      22
(B, B, C)      13
(C, A, B)       8
(C, A, A)       2
(B, C, C)       1
(C, C, A)       1
(A, A, A)       1


In [9]:
cumulative_election.run_election()

Candidate     Status  Round
        B    Elected      1
        A    Elected      1
        C Eliminated      1

Double check the computation above!


Now what happens in the case of ties? As currently coded, `votekit` takes all the tied candidates and randomly permutes them.

In [13]:
tied_cumulative_profile = PreferenceProfile(ballots = [CumulativeBallot(multi_votes=["A", "B", "C"])],
                                       candidates = ["A", "B", "C"])

In [15]:
cumulative_election = Cumulative(profile = tied_cumulative_profile,
                                 seats = 2)
cumulative_election.run_election()

There was a tie between ['A', 'B', 'C'].
The tie was broken by randomly permuting the candidates: ['C', 'A', 'B'].


Candidate     Status  Round
        C    Elected      1
        A    Elected      1
        B Eliminated      1